## DNN 모듈 

### 1.1 OpenCV DNN 모듈
- OpenCV DNN 모듈
    - 미리 학습된 딥러닝 모델을 이용하여 실행(forward pass, inference) 하는 기능
    - 학습은 지원하지 않음
    - OpenCV 3.3 버전부터 기본 기능으로 제공
    - OpenCV 4.3 버전부터 GPU(CUDA) 지원 (소스 코드 직접 빌드 필요)
    - 참고 : https://github.com/opencv/opencv/wiki/Deep-Learning-in-OpenCV
- 지원하는 딥러닝 프레임워크
    - Caffe, Tensorflow, Pytorch, Darknet, ONNX

### 1.2 네트워크 불러오기
- **cv2.dnn.readNet(model, config=None, framework=None) -> retval**
    - model : 훈련된 가중치를 저장하고 있는 이진 파일 이름
    - config : 네트워크 구성을 저장하고 있는 텍스트 파일 이름
    - framework : 명시적인 딥러닝 프레임워크 이름
    - retval : cv2.dnn_Net 클래스 객체

### 1.3 네트워크 입력 블롭(blob) 만들기
- **cv2.dnn.blobFromImage(image, scalefactor=None, size=None, mean=None,
                          swapRB=None, crop=None, ddepth=None) -> retval**
    - image : 입력 영상
    - scalefactor : 입력 영상 픽셀 값에 곱할 값. 기본값은 1
    - size : 출력 영상의 크기. 기본값은 (0, 0)
    - mean : 입력 영상 각 채널에서 뺄 평균 값. 기본값은 (0, 0, 0, 0)
    - swapRB : R과 B채널을 서로 바꿀 것인지를 결정하는 플래그. 기본값은 False
    - crop : 크롭(crop) 수행 여부. 기본값은 False
    - ddepth : 출력 블롭의 깊이. CV_32F 또는 CV_8U. 기본값은 CV_32F
    - retval : 영상으로부터 구한 블롭 객체.
        - numpy.ndarray.shape = (N,C,H,W), dtype = numpy.float32

### 1.4 네트워크 입력 설정
- **cv2.dnn_Net.setInput(blob, name=None, scalefactor=None, mean=None) -> None**
    - blob : 블롭 객체
    - name : 입력 레이어 이름
    - scalefactor : 추가적으로 픽셀 값에 곱할 값
    - mean : 추가적으로 픽셀 값에서 뺄 평균 값

### 1.5 네트워크 순방향추론
- **cv2.dnn_Net.forward(outputName=None) -> retval**
- **cv2.dnn_Net.forward(outputNames=None, outputBlobs=None) -> outputBlobs**
    - outputName : 출력 레이어 이름
    - retval : 지정한 레이어의 출력 블롭. 네트워크마다 다르게 결정됨
    - outputNames : 출력 레이어 이름 리스트
    - outputBlobs : 지정한 레이어의 출력 블롭 리스트

### 1.6 mnist

In [1]:
from tensorflow.keras import datasets

mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

### 1.7 모델 구축

In [2]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                  padding='same', activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 7, 7, 64)          0         
                                                                 
 flatten (Flatten)           (None, 3136)              0

### 1.8 학습

In [3]:
import time

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

start_time = time.time()
hist = model.fit(X_train, y_train, epochs=5, verbose=1, batch_size=1000,
                 validation_data = (X_test, y_test))
print('fit time : ', time.time() - start_time)

Epoch 1/5
60/60 [==============================] - 21s 346ms/step - loss: 0.4514 - accuracy: 0.8693 - val_loss: 0.1132 - val_accuracy: 0.9661
Epoch 2/5
60/60 [==============================] - 21s 351ms/step - loss: 0.1014 - accuracy: 0.9696 - val_loss: 0.0617 - val_accuracy: 0.9818
Epoch 3/5
60/60 [==============================] - 21s 356ms/step - loss: 0.0682 - accuracy: 0.9793 - val_loss: 0.0435 - val_accuracy: 0.9855
Epoch 4/5
60/60 [==============================] - 23s 382ms/step - loss: 0.0545 - accuracy: 0.9833 - val_loss: 0.0413 - val_accuracy: 0.9859
Epoch 5/5
60/60 [==============================] - 23s 392ms/step - loss: 0.0449 - accuracy: 0.9865 - val_loss: 0.0328 - val_accuracy: 0.9888
fit time :  110.05416107177734


### 1.9 test accuracy 98%

In [4]:
score = model.evaluate(X_test, y_test)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

313/313 [==============================] - 3s 8ms/step - loss: 0.0328 - accuracy: 0.9888
Test loss : 0.032845389097929
Test accuracy : 0.9887999892234802


### 1.10 모델 저장

In [5]:
# model.save('C:/seonwook/DL/OpenCV/mnist_cnn')

### 1.11 pb파일 생성을 위한 변환

In [6]:
import tensorflow as tf

In [7]:
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

In [8]:
# Get frozen ConcreteFunction
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

frozen_func = convert_variables_to_constants_v2(full_model)
# frozen_func.graph.as_graph_def() 

- 모델의 구조와 웨이트가 모두 저장

### 1.12 레이어 확인

In [9]:
layers = [op.name for op in frozen_func.graph.get_operations()]
# print('-' * 50)
# print('Frozen model layers: ')
# for layer in layers:
#     print(layer)

# print('-' * 50)
# print('Frozen model inputs: ')
# print(frozen_func.inputs)
# print('Frozen model outputs: ')
# print(frozen_func.outputs)

### 1.13 pb로 저장

In [10]:
# Save frozen graph from frozen ConcreteFunction to hard drive
# tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
#                   logdir='./frozen_models',
#                   name='frozen_graph.pb',
#                   as_text=False)

### 1.14 opencv에서 추론

In [2]:
import sys
import numpy as np
import cv2

oldx, oldy = -1, -1

def on_mouse(event, x, y, flags, _):
    global oldx, oldy

    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y
    
    elif event == cv2.EVENT_LBUTTONUP:
        oldx, oldy = -1, -1
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 40, cv2.LINE_AA)
            oldx, oldy = x, y
            cv2.imshow('img', img)

net = cv2.dnn.readNetFromTensorflow('./frozen_models/frozen_graph.pb')

if net.empty():
    print('Network load failed')
    sys.exit()

img = np.zeros((400, 400), np.uint8)

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)

while True:
    c = cv2.waitKey()

    if c == 27:
        break
    elif c == ord(' '):
        blob = cv2.dnn.blobFromImage(img, 1/255., (28, 28))
        net.setInput(blob)
        prob = net.forward()

        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)
        digit = maxLoc[0]

        print(f'{digit} ({maxVal * 100:4.2f}%)')

        img.fill(0)
        cv2.imshow('img', img)

cv2.destroyAllWindows()

2 (99.93%)
7 (82.44%)
7 (63.07%)
5 (100.00%)
4 (99.88%)


### 1.15 모멘트
- 모멘트를 이용해서 그림의 중심을 찾을 수 있음

In [3]:
import cv2
src = cv2.imread("digits_test.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
M = cv2.moments(gray)
print(M)
cX = int(M['m10'] / M['m00'])
cY = int(M['m01'] / M['m00'])
cv2.circle(src, (cX, cY), 5, (255, 0, 0), -1)
 
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

{'m00': 27864765.0, 'm10': 11708252630.0, 'm01': 5540303468.0, 'm20': 5482973170932.0, 'm11': 2330559803482.0, 'm02': 1417073617236.0, 'm30': 2714153882789594.0, 'm21': 1089720437669904.0, 'm12': 607718273485288.0, 'm03': 408657928000928.0, 'mu20': 563384592025.919, 'mu11': 2627999635.489337, 'mu02': 315504574126.0788, 'mu30': -63136121678841.586, 'mu21': -2658193233792.011, 'mu12': 11245425539295.326, 'mu03': 1441141913370.1262, 'nu20': 0.0007255948570512897, 'nu11': 3.3846559647410453e-06, 'nu02': 0.00040634497215981397, 'nu30': -1.5404210288365443e-05, 'nu21': -6.485569032689687e-07, 'nu12': 2.7437051117999384e-06, 'nu03': 3.51615723275736e-07}


- 모멘트 (이미지의 경우)
    - 이미지의 픽셀 값이 0이 아닌 값은 모두 1의 값으로 변경해 모멘트를 계산
    - 모멘트 함수를 통해 면적, 평균, 분산 등을 간단하게 계산할 수 있음
    - 중심점 구하는 공식
        - $ \overline{x} = \frac{m_{10}}{m_{00}}, \overline{y} = \frac{m_{01}}{m_{00}}$



        

In [6]:
import sys
import numpy as np
import cv2

oldx, oldy = -1, -1

def on_mouse(event, x, y, flags, _):
    global oldx, oldy

    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y
    
    elif event == cv2.EVENT_LBUTTONUP:
        oldx, oldy = -1, -1
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 40, cv2.LINE_AA)
            oldx, oldy = x, y
            cv2.imshow('img', img)

def norm_digit(img):
    m = cv2.moments(img)
    print(m)
    cx = m['m10'] / m['m00']
    cy = m['m01'] / m['m00']
    h, w = img.shape[:2]
    aff = np.array([[1, 0, w/2 - cx], [0, 1, h/2 - cy]], dtype=np.float32)
    dst = cv2.warpAffine(img, aff, (0, 0))
    return dst

net = cv2.dnn.readNet('./frozen_models/frozen_graph.pb')

if net.empty():
    print('Network load failed')
    sys.exit()

img = np.zeros((400, 400), np.uint8)

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)

while True:
    c = cv2.waitKey()

    if c == 27:
        break
    elif c == ord(' '):
        blob = cv2.dnn.blobFromImage(norm_digit(img), 1/255., (28, 28))
        net.setInput(blob)
        prob = net.forward()

        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)
        digit = maxLoc[0]

        print(f'{digit} ({maxVal * 100:4.2f}%)')

        img.fill(0)
        cv2.imshow('img', img)

cv2.destroyAllWindows()

{'m00': 8019652.0, 'm10': 1287200513.0, 'm01': 1520047179.0, 'm20': 247868201363.0, 'm11': 248935872883.0, 'm02': 324460355481.0, 'm30': 54061671719993.0, 'm21': 48993378080417.0, 'm12': 54018531456523.0, 'm03': 75334409790903.0, 'mu20': 41265076854.921196, 'mu11': 4959512238.797126, 'mu02': 36350169853.76101, 'mu30': 1030825475777.053, 'mu21': 420306126465.50146, 'mu12': 60713686013.18706, 'mu03': 56455793523.58839, 'nu20': 0.0006416107206567965, 'nu11': 7.711305695196683e-05, 'nu02': 0.0005651912089698955, 'nu30': 5.6597381025015525e-06, 'nu21': 2.3076870474882663e-06, 'nu12': 3.333479528269529e-07, 'nu03': 3.099700319994036e-07}
4 (99.99%)
{'m00': 6114319.0, 'm10': 1268005205.0, 'm01': 1193259224.0, 'm20': 276430879761.0, 'm11': 254906896246.0, 'm02': 262410457866.0, 'm30': 62922858572987.0, 'm21': 57043534970262.0, 'm12': 57489486994654.0, 'm03': 62570928934994.0, 'mu20': 13468283287.526495, 'mu11': 7445338066.542739, 'mu02': 29536187540.705853, 'mu30': 9637823190.765766, 'mu21': 7

---

## 사전 학습 모델 활용

### 2.1 GoogLeNet
- 2014년 ILSVRC(ImageNet Large Scale Visual Revognition Competition) 영상 인식 분야 1위
    - 1000개의 카테고리, 120만개의 훈련 영상, 15만개의 테스트 영상
- 입력 : 224x224, BGR 컬러 영상, 평균 값 = (104, 117, 123)
- 출력 : 1x1000 행렬, 1000개 클래스에 대한 확률 값

###  2.2 모델 받기
- Caffe model https://github.com/BVLC/caffe
- Model : http://dl.caffe.berkeleyvision.org/bvlc_googlenet.caffemodel
- 설정파일 : https://github.com/BVLC/caffe/blob/master/models/bvlc_googlenet/deploy.prototxt
- ONNX model : https://github.com/onnx/models
- ONNX 모델 파일 : https://github.com/onnx/models/tree/master/vision/classification/inception_and_googlenet/googlenet
- GoogLeNet 클래스 이름 파일 : https://github.com/opencv/opencv/blob/4.1.0/samples/data/dnn/classification_classes_ILSVRC2012.txt

In [3]:
import sys
import numpy as np
import cv2

# 입력 영상 불러오기

filename = 'Model_S.jpg'

if len(sys.argv) > 1:
    filename = sys.argv[1]

img = cv2.imread(filename)

if img is None:
    print('Image load failed!')
    sys.exit()

# 네트워크 불러오기

# Caffe
model = 'googlenet/bvlc_googlenet.caffemodel'
config = 'googlenet/deploy.prototxt'

# ONNX
#model = 'googlenet/inception-v1-9.onnx'
#config = ''

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Network load failed!')
    sys.exit()

# 클래스 이름 불러오기

classNames = None
with open('googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

# 추론
blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123))
net.setInput(blob)
prob = net.forward()

# 추론 결과 확인 & 화면 출력
out = prob.flatten()
classId = np.argmax(out)
confidence = out[classId]

text = f'{classNames[classId]} ({confidence * 100:4.2f}%)'
cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

Image load failed!


SystemExit: 